In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.sklearn_transformers import *
from feature_engineering.store_features import *
from feature_engineering.bus_stop_features import *
from feature_engineering.utils import *
from feature_engineering.demographic_features import *

stop_importance_levels = ['Mangler viktighetsnivå',
                          'Standard holdeplass',
                          'Lokalt knutepunkt',
                          'Nasjonalt knutepunkt',
                          'Regionalt knutepunkt',
                          'Annen viktig holdeplass']
store_types = ['lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc']
geo_groups = ['grunnkrets_id','t_district','municipality_name']

In [2]:
stores_train_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv"))
stores_test_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv"))
stores_extra_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv"))

income_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv"))
age_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv"))
household_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv"))
grunnkrets_df = combine_keys(set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")))
plaace_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/plaace_hierarchy.csv")

bus_stops_df = bus_stops_lat_lon(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv"))


/Users/nwong/Workspace/Projects/tdt4173_project/src/feature_engineering/bus_stop_features.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bus_stops_df[['lon', 'lat']] = bus_stops_df['lng_lat'].str.split(


In [6]:
store_count(stores_train_df.merge(plaace_df, on="plaace_hierarchy_id", how="left").merge(grunnkrets_df, on="grunnkrets_id", how="left"), ['municipality_name', 'lv2_desc'], "count")
# population_count_grouped_by_geo_group(stores_train_df, age_dist, grunnkrets_df, geo_groups)

,municipality_name,lv2_desc,count
0,Agdenes,Food and drinks,1
1,Alstahaug,Beauty and wellbeing,3
2,Alstahaug,Clothing stores,3
3,Alstahaug,Food and drinks,5
4,Alstahaug,Gyms,2
...,...,...,...
3104,Øystre Slidre,Gyms,1
3105,Øystre Slidre,Other Retail,1
3106,Øystre Slidre,Pubs and bars,3
3107,Øystre Slidre,Restaurant,5


In [ ]:
def new_kmeans(stores_train_merged, stores_test_merged, groups):
    train_list = []
    test_list = []
    for group in groups:
        _stores_train_merged = stores_train_merged[stores_train_merged['lv2_desc'] == group]
        _stores_test_merged = stores_test_merged[stores_test_merged['lv2_desc'] == group]
        kmeans = KMeans(n_clusters=1, init='k-means++')
        kmeans.fit(_stores_train_merged[['lat', 'lon']], sample_weight=_stores_train_merged['revenue'])
        _stores_train_merged[f'{group}_cluster_label'] = kmeans.predict(_stores_train_merged[['lat', 'lon']])
        _stores_test_merged[f'{group}_cluster_label'] = kmeans.predict(_stores_test_merged[['lat', 'lon']])
        
        train_list.append(_stores_train_merged[['store_id', f'{group}_cluster_label']])
        test_list.append(_stores_test_merged[['store_id', f'{group}_cluster_label']])
    
    return pd.concat(train_list), pd.concat(test_list)
    


In [ ]:
groups = plaace_df.lv2_desc.unique()

stores_train_merged = stores_train_df.merge(plaace_df, on="plaace_hierarchy_id", how="inner")
stores_test_merged = stores_test_df.merge(plaace_df, on="plaace_hierarchy_id", how="inner")
new_kmeans(stores_train_merged, stores_test_merged, groups)

In [ ]:
stores_train_df